# Tiền xử lý dữ liệu nâng cao cho bài toán dự đoán giá cước Taxi (Google Colab Version)

## Hướng dẫn sử dụng với Google Colab:
1. File này đã được đồng bộ với GitHub tại: https://github.com/Masterevan-20/taxi_fare
2. Để chạy trên Colab:
   - Mở Google Colab
   - Chọn File > Open Notebook > GitHub
   - Nhập URL repository và chọn notebook này
3. Đảm bảo tải file dữ liệu `uber.csv` lên Colab hoặc sử dụng Google Drive

# Tiền xử lý dữ liệu nâng cao cho bài toán dự đoán giá cước Taxi

File này thực hiện các phương pháp tiền xử lý nâng cao, tập trung vào:
1. Phân tích khám phá dữ liệu chi tiết (EDA)
2. Xử lý dữ liệu thiếu và ngoại lai thông minh
3. Feature engineering nâng cao
4. Lựa chọn feature với nhiều phương pháp khác nhau

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
import holidays
import folium
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu
print('Đọc dữ liệu từ file...')
df = pd.read_csv('uber.csv')
print(f'Kích thước dữ liệu: {df.shape}')

Đọc dữ liệu từ file...
Kích thước dữ liệu: (200000, 9)
Kích thước dữ liệu: (200000, 9)


: 

In [ ]:
# Code để mount Google Drive (bỏ comment nếu sử dụng Google Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# Nếu file uber.csv nằm trong Google Drive, thay đổi đường dẫn tương ứng
# df = pd.read_csv('/content/drive/MyDrive/path_to_your_file/uber.csv')

# Nếu upload trực tiếp file uber.csv lên Colab, giữ nguyên dòng dưới
df = pd.read_csv('uber.csv')
print(f'Kích thước dữ liệu: {df.shape}')

## 1. Phân tích khám phá dữ liệu nâng cao (EDA)

In [ ]:
# Phân tích thống kê chi tiết
print('\nThông tin cơ bản về dữ liệu:')
print(df.info())

print('\nThống kê mô tả dữ liệu số:')
print(df.describe())

# Kiểm tra giá trị thiếu
print('\nPhân tích dữ liệu thiếu:')
missing_data = pd.DataFrame({
    'Số lượng thiếu': df.isnull().sum(),
    'Phần trăm thiếu': (df.isnull().sum() / len(df)) * 100
})
print(missing_data[missing_data['Số lượng thiếu'] > 0])

# Phân tích phân phối của các biến số
plt.figure(figsize=(15, 10))
for i, column in enumerate(df.select_dtypes(include=['float64', 'int64']).columns, 1):
    plt.subplot(3, 3, i)
    sns.histplot(df[column], kde=True)
    plt.title(f'Phân phối của {column}')
plt.tight_layout()
plt.show()


Thông tin cơ bản về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB
None

Thống kê mô tả dữ liệu số:
         Unnamed: 0    fare_amount  pickup_longitude  pickup_latitude  \
count  2.000000e+05  200000.000000     200000.000000    200000.000000   
mean   2.771250e+07      11.359955        -72.527638        39.935885   
std   

KeyboardInterrupt: 

### 1.1 Phân tích không gian địa lý

In [ ]:
# Phân tích phân bố địa lý của các chuyến đi
def create_pickup_heatmap(df, num_points=1000):
    # Lấy mẫu ngẫu nhiên để tránh quá tải
    sample_df = df.sample(n=min(num_points, len(df)))
    
    # Tạo bản đồ với tọa độ trung tâm của New York
    nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=11)
    
    # Thêm heatmap
    pickup_points = [[row['pickup_latitude'], row['pickup_longitude']] 
                     for idx, row in sample_df.iterrows()]
    folium.HeatMap(pickup_points).add_to(nyc_map)
    
    return nyc_map

pickup_heatmap = create_pickup_heatmap(df)
pickup_heatmap.save('pickup_heatmap.html')
print('Đã tạo bản đồ nhiệt điểm đón khách tại pickup_heatmap.html')

### 1.2 Phân tích thời gian

In [ ]:
# Chuyển đổi cột pickup_datetime sang định dạng datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Trích xuất các đặc trưng thời gian
df['hour'] = df['pickup_datetime'].dt.hour
df['day'] = df['pickup_datetime'].dt.day
df['month'] = df['pickup_datetime'].dt.month
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['week_of_year'] = df['pickup_datetime'].dt.isocalendar().week

# Phân tích xu hướng theo thời gian
plt.figure(figsize=(20, 10))

# Giá trung bình theo giờ
plt.subplot(2, 2, 1)
hourly_fare = df.groupby('hour')['fare_amount'].mean()
sns.lineplot(x=hourly_fare.index, y=hourly_fare.values)
plt.title('Giá trung bình theo giờ')

# Giá trung bình theo ngày trong tuần
plt.subplot(2, 2, 2)
daily_fare = df.groupby('day_of_week')['fare_amount'].mean()
sns.barplot(x=daily_fare.index, y=daily_fare.values)
plt.title('Giá trung bình theo ngày trong tuần')

# Giá trung bình theo tháng
plt.subplot(2, 2, 3)
monthly_fare = df.groupby('month')['fare_amount'].mean()
sns.lineplot(x=monthly_fare.index, y=monthly_fare.values)
plt.title('Giá trung bình theo tháng')

# Box plot theo ngày trong tuần
plt.subplot(2, 2, 4)
sns.boxplot(x='day_of_week', y='fare_amount', data=df)
plt.title('Phân phối giá theo ngày trong tuần')

plt.tight_layout()
plt.show()

## 2. Xử lý dữ liệu thiếu và ngoại lai

In [ ]:
def remove_outliers_iqr(df, columns, k=1.5):
    """Loại bỏ ngoại lai dùng phương pháp IQR"""
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        df_clean = df_clean[
            (df_clean[col] >= Q1 - k * IQR) & 
            (df_clean[col] <= Q3 + k * IQR)
        ]
    return df_clean

# Xử lý ngoại lai cho các cột số
numeric_columns = ['fare_amount', 'trip_distance', 'pickup_latitude', 
                  'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']

# Loại bỏ các giá trị không hợp lệ
df_cleaned = df[
    (df['fare_amount'] > 0) & 
    (df['trip_distance'] > 0) &
    (df['pickup_latitude'].between(40, 42)) &
    (df['pickup_longitude'].between(-75, -73)) &
    (df['dropoff_latitude'].between(40, 42)) &
    (df['dropoff_longitude'].between(-75, -73))
]

# Loại bỏ ngoại lai
df_cleaned = remove_outliers_iqr(df_cleaned, numeric_columns)

print(f'Số lượng bản ghi sau khi xử lý ngoại lai: {len(df_cleaned)}')

# Xử lý dữ liệu thiếu bằng KNN Imputer
imputer = KNNImputer(n_neighbors=5)
df_cleaned[numeric_columns] = imputer.fit_transform(df_cleaned[numeric_columns])

# Kiểm tra lại dữ liệu thiếu
print('\nKiểm tra dữ liệu thiếu sau khi xử lý:')
print(df_cleaned.isnull().sum())

## 3. Feature Engineering nâng cao

In [ ]:
# Thêm các đặc trưng mới

# 1. Đặc trưng thời gian chu kỳ
df_cleaned['hour_sin'] = np.sin(2 * np.pi * df_cleaned['hour']/24)
df_cleaned['hour_cos'] = np.cos(2 * np.pi * df_cleaned['hour']/24)
df_cleaned['day_sin'] = np.sin(2 * np.pi * df_cleaned['day']/31)
df_cleaned['day_cos'] = np.cos(2 * np.pi * df_cleaned['day']/31)

# 2. Đặc trưng khoảng cách
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Bán kính trái đất (km)
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

df_cleaned['haversine_distance'] = haversine_distance(
    df_cleaned['pickup_latitude'],
    df_cleaned['pickup_longitude'],
    df_cleaned['dropoff_latitude'],
    df_cleaned['dropoff_longitude']
)

# 3. Đặc trưng thời gian
df_cleaned['is_weekend'] = df_cleaned['day_of_week'].isin([5, 6]).astype(int)
df_cleaned['is_rush_hour'] = df_cleaned['hour'].isin([7, 8, 9, 16, 17, 18]).astype(int)

# 4. Đặc trưng địa lý
df_cleaned['manhattan_distance'] = (
    np.abs(df_cleaned['dropoff_latitude'] - df_cleaned['pickup_latitude']) +
    np.abs(df_cleaned['dropoff_longitude'] - df_cleaned['pickup_longitude'])
)

# 5. Thêm ngày lễ
us_holidays = holidays.US()
df_cleaned['is_holiday'] = df_cleaned['pickup_datetime'].dt.date.map(
    lambda x: x in us_holidays
).astype(int)

# 6. Tốc độ ước tính
df_cleaned['estimated_speed'] = df_cleaned['trip_distance'] / \
    (df_cleaned['fare_amount'] / 2.5)  # Giả sử 2.5$ là giá cước cho mỗi mile

print('Các đặc trưng mới đã được thêm vào:')
print(df_cleaned.columns.tolist())

## 4. Lựa chọn đặc trưng

In [ ]:
# Chuẩn bị dữ liệu cho feature selection
feature_columns = [
    'passenger_count', 'trip_distance', 'hour_sin', 'hour_cos',
    'day_sin', 'day_cos', 'is_weekend', 'is_rush_hour',
    'haversine_distance', 'manhattan_distance', 'is_holiday',
    'estimated_speed'
]

X = df_cleaned[feature_columns]
y = df_cleaned['fare_amount']

# 1. Univariate feature selection
k_best = SelectKBest(score_func=f_regression, k=8)
k_best.fit(X, y)
univariate_scores = pd.DataFrame({
    'Feature': feature_columns,
    'Score': k_best.scores_
}).sort_values('Score', ascending=False)

print('Top features theo f_regression:')
print(univariate_scores)

# 2. Recursive Feature Elimination
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=8)
rfe.fit(X, y)

rfe_scores = pd.DataFrame({
    'Feature': feature_columns,
    'Selected': rfe.support_,
    'Rank': rfe.ranking_
}).sort_values('Rank')

print('\nFeatures được chọn bởi RFE:')
print(rfe_scores)

# 3. Feature Importance từ Random Forest
rf.fit(X, y)
importance_scores = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print('\nFeature importance từ Random Forest:')
print(importance_scores)

# Visualize feature importance
plt.figure(figsize=(12, 6))
sns.barplot(data=importance_scores, x='Importance', y='Feature')
plt.title('Feature Importance từ Random Forest')
plt.show()

# 4. PCA Analysis
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA()
pca.fit(X_scaled)

# Plot explained variance ratio
plt.figure(figsize=(10, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('PCA Analysis')
plt.show()

# Chọn các features tốt nhất dựa trên kết hợp các phương pháp
def get_best_features(importance_df, univariate_df, rfe_df, n_features=8):
    # Normalize scores
    importance_df['norm_score'] = importance_df['Importance'] / importance_df['Importance'].max()
    univariate_df['norm_score'] = univariate_df['Score'] / univariate_df['Score'].max()
    
    # Combine scores
    final_scores = pd.DataFrame()
    final_scores['Feature'] = feature_columns
    final_scores['combined_score'] = (
        importance_df['norm_score'] +
        univariate_df['norm_score'] +
        (rfe_df['Selected'].astype(int) * 0.5)  # Giving less weight to RFE
    )
    
    return final_scores.nlargest(n_features, 'combined_score')['Feature'].tolist()

best_features = get_best_features(importance_scores, univariate_scores, rfe_scores)
print('\nCác features tốt nhất được chọn:')
print(best_features)

# Lưu dữ liệu đã xử lý
df_final = df_cleaned[best_features + ['fare_amount']]
df_final.to_csv('uber_cleaned.csv', index=False)
print('\nĐã lưu dữ liệu đã xử lý vào uber_cleaned.csv')

## 5. Kiểm tra chất lượng dữ liệu sau xử lý

In [ ]:
# Kiểm tra phân phối của các features đã chọn
plt.figure(figsize=(15, 10))
for i, feature in enumerate(best_features, 1):
    plt.subplot(3, 3, i)
    sns.histplot(df_final[feature], kde=True)
    plt.title(f'Phân phối của {feature}')
plt.tight_layout()
plt.show()

# Kiểm tra tương quan
plt.figure(figsize=(12, 8))
sns.heatmap(df_final.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Ma trận tương quan của các features đã chọn')
plt.show()

# Thống kê mô tả cuối cùng
print('\nThống kê mô tả của dữ liệu sau xử lý:')
print(df_final.describe())

# Kiểm tra phân phối của biến mục tiêu
plt.figure(figsize=(10, 6))
sns.histplot(df_final['fare_amount'], kde=True)
plt.title('Phân phối của giá cước sau xử lý')
plt.show()